## (a)

In [1]:
import pandas as pd
import numpy as np
import pyomo.environ as pe
solver = pe.SolverFactory('gurobi')

In [2]:
train_data = pd.read_csv('train.csv')
train_data

,Unnamed: 0,X1,X2,Y
0,0,8.820262,2.000786,18.531508
1,1,4.893690,11.204466,47.159228
2,2,9.337790,-4.886389,30.416522
3,3,4.750442,-0.756786,6.986012
4,4,-0.516094,2.052993,14.764671
...,...,...,...,...
995,995,6.713110,-3.699679,9.249274
996,996,6.575688,-1.617287,15.047041
997,997,0.989141,0.488754,10.834011
998,998,7.007617,0.792169,6.349938


In [3]:
x_train = train_data[['X1','X2']]
y_train = train_data['Y']
x_train = np.array(x_train)
y_train = np.array(y_train)

In [4]:
test_data = pd.read_csv('test.csv')
test_data

,Unnamed: 0,X1,X2,Y
0,0,-7.664605,-8.559851,34.143676
1,1,0.230675,-4.791872,22.535028
2,2,-0.404058,-3.519295,16.393405
3,3,-3.853922,-2.404227,4.369055
4,4,3.517928,4.645726,21.562645
...,...,...,...,...
495,495,-1.702800,-6.516100,28.864879
496,496,2.333753,0.805532,3.636546
497,497,1.600160,10.395883,57.221907
498,498,-4.537330,-0.962021,3.971413


In [5]:
x_test = test_data[['X1','X2']]
y_test = test_data['Y']
y_test = np.array(y_test)
x_test = np.array(x_test)

In [6]:
from sklearn.ensemble import RandomForestRegressor
import scipy.stats as stats
n= 1000
T = 500
c, p = 0.5, 1

In [7]:
forest_train = RandomForestRegressor(max_depth=5)
forest_train.fit(x_train, y_train)
y_pred = forest_train.predict(x_test)

In [8]:
from tqdm.notebook import tqdm

z_a = np.zeros(T)

for t in tqdm(range(T)):
    model_a = pe.ConcreteModel() 
    model_a.z = pe.Var(domain=pe.NonNegativeReals)
    model_a.s = pe.Var(domain=pe.Reals)
    model_a.cost = pe.Objective(expr=c*model_a.z-p*model_a.s)
    model_a.constraint1 = pe.Constraint(expr=model_a.s <= model_a.z)
    model_a.constraint2 = pe.Constraint(expr=model_a.s <= y_pred[t])
    solver.solve(model_a)
    z_a[t] = model_a.z()

  0%|          | 0/500 [00:00<?, ?it/s]

    solver gurobi


ApplicationError: No executable found for solver 'gurobi'

In [9]:
def cost_func(z, y, c=0.5, p=1):
    return c*z - p*np.minimum(z, y)


perf_PTO = cost_func(z_a, y_test)

print("Point-prediction-driven optimization:")
print("Expected cost: {0:0.4f}".format(perf_PTO.mean()))

Point-prediction-driven optimization:
Expected cost: 0.0000


In [10]:
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from scipy.stats import uniform
from xgboost import XGBRegressor


# instantiate an XGBRegressor with default hyperparameter settings
xgb = XGBRegressor(objective = 'reg:squarederror')

# compute a baseline to beat with hyperparameter optimization 
# average -MSE on the 5 validation sets
baseline = cross_val_score(xgb, x_train, y_train, scoring = 'neg_mean_squared_error').mean()
param_dist = {
    "learning_rate": uniform(0, 1),
    "gamma": uniform(0, 5),
    "max_depth": range(1, 50),
    "n_estimators": range(1, 300),
    "min_child_weight": range(1, 10)}

rs = RandomizedSearchCV(
    xgb, param_distributions = param_dist, 
    scoring = 'neg_mean_squared_error', n_iter = 25)

# run random search for 25 iterations
rs.fit(x_train, y_train)

RandomizedSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, gamma=None,
                                          gpu_id=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None, max_bin=None,
                                          max_ca...
                                          reg_alpha=None, reg_lambda=None, ...),
                   n_iter=25,
                   param_distributions={'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000015E61B32490>,
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000015E61AAA4C0>,
                                        'max_depth': range(1, 50),
                                        'min_child_weight': range(1, 10),
                                        'n_estimators': range(1, 300)},
                   scoring='neg_mean_squared_error')

In [11]:
import GPy
import GPyOpt
from GPyOpt.methods import BayesianOptimization
bds = [
    {'name': 'learning_rate', 'type': 'continuous', 'domain': (0, 1)},
    {'name': 'gamma', 'type': 'continuous', 'domain': (0, 5)},
    {'name': 'max_depth', 'type': 'discrete', 'domain': (1, 50)},
    {'name': 'n_estimators', 'type': 'discrete', 'domain': (1, 300)},
    {'name': 'min_child_weight', 'type': 'discrete', 'domain': (1, 10)}]

# optimization objective 
def cv_score(parameters):
    parameters = parameters[0]
    score = cross_val_score(
        XGBRegressor(
            objective = 'reg:squarederror',
            learning_rate = parameters[0],
            gamma = int(parameters[1]),
            max_depth = int(parameters[2]),
            n_estimators = int(parameters[3]),
            min_child_weight = parameters[4]), 
            x_train, y_train, scoring = 'neg_mean_squared_error').mean()
    score = np.array(score)
    return score

# by default, initial_design_numdata = 5
optimizer = BayesianOptimization(
    f = cv_score, 
    domain = bds,
    model_type = 'GP',
    acquisition_type = 'EI',
    acquisition_jitter = 0.05,
    exact_feval = True, 
    maximize = True)

# only 20 iterations + 5 initial random points
optimizer.run_optimization(max_iter = 20)

In [12]:
rs.best_params_

{'gamma': 0.894992562991383,
 'learning_rate': 0.21235329027534477,
 'max_depth': 7,
 'min_child_weight': 5,
 'n_estimators': 101}

In [13]:
y_rs = np.maximum.accumulate(rs.cv_results_['mean_test_score'])
y_bo = np.maximum.accumulate(-optimizer.Y).ravel()

print(f'Baseline neg. MSE = {baseline:.2f}')
print(f'Random search neg. MSE = {y_rs[-1]:.2f}')
print(f'Bayesian optimization neg. MSE = {y_bo[-1]:.2f}')

Baseline neg. MSE = -31.85
Random search neg. MSE = -31.98
Bayesian optimization neg. MSE = -38.02


In [14]:
leaves=forest_train.apply(x_train)

In [ ]:
z_tree = np.zeros(T)

for t in tqdm(range(T)):
    tofs=[]

    w = np.zeros(n) 
    model_tree = pe.ConcreteModel() 
    model_tree.z = pe.Var(domain=pe.NonNegativeReals)
    model_tree.s = pe.Var(range(n), domain=pe.Reals)
    leaf = forest_train.apply([x_test[t, :]])[0]
    for i in range(len(leaves)):
        tot = sum(leaf == leaves[i])
        tofs.append(tot) 
    for i in range(n):
        for j in range(leaves.shape[1]):
            w[i]+=(1/n)*((leaf[j]== leaves[i][j]) / tofs[i])
    
    model_tree.cost = pe.Objective(expr=sum(w[i]*(c*model_tree.z-p*model_tree.s[i]) for i in range(n)))
    model_tree.constraint1 = pe.Constraint(range(n), rule=
                                          lambda mod, i: mod.s[i] <= mod.z)
    model_tree.constraint2 = pe.Constraint(range(n), rule=
                                          lambda mod, i: mod.s[i] <= y_train[i])
    #solver.solve(model_tree)
    z_tree[t] = model_tree.z()

In [17]:
perf_PTO = cost_func(z_tree, y_test)

print("Point-prediction-driven optimization:")
print("Expected cost: {0:0.4f}".format(perf_PTO.mean()))

Point-prediction-driven optimization:
Expected cost: nan


In [ ]:
def kern_Gaus(u):
    return np.exp(-np.sum(u**2))/np.sqrt(2*np.pi)
z_kern = np.zeros(T)
bandwidth = 2 # perf-sensitive parameter, h

for t in tqdm(range(T)):   
    w = np.zeros(n)
    model_kern = pe.ConcreteModel() 
    model_kern.z = pe.Var(domain=pe.NonNegativeReals)
    model_kern.s = pe.Var(range(n), domain=pe.Reals)    
    
    for i in range(n):
        w[i] = kern_Gaus((x_test[t,:] - x_train[i,:])/bandwidth) 
    
    model_kern.cost = pe.Objective(expr=sum(w[i]*(c*model_kern.z-p*model_kern.s[i]) for i in range(n)))
    model_kern.constraint1 = pe.Constraint(range(n), rule=
                                          lambda mod, i: mod.s[i] <= mod.z)
    model_kern.constraint2 = pe.Constraint(range(n), rule=
                                          lambda mod, i: mod.s[i] <= y_train[i])
    solver.solve(model_kern)
    z_kern[t] = model_kern.z()

In [19]:
perf_PTO = cost_func(z_kern, y_test)

print("Point-prediction-driven optimization:")
print("Expected cost: {0:0.4f}".format(perf_PTO.mean()))

Point-prediction-driven optimization:
Expected cost: 0.0000
